In [8]:
import pandas as pd
import stanza
from text_mining.text_processing.text2bow import Text2BowTransformerPhrases
from text_mining.text_processing.tokenizer import Tokenizer

In [3]:
directory = '~/PycharmProjects/tfm_hugopobil'
df = pd.read_csv(f'{directory}/data/tweets_nlp_modelling.csv')
btc_usd_grouped = pd.read_csv(f'{directory}/data/btc_usd_grouped.csv')
df = df.set_index('date_clean')

In [9]:
stanza.download('en')

In [15]:
def head_body2bow(df, language='en'):
    X = df['cleaned_tweets']
    tokenizer = Tokenizer(tokenizer_model='stanza', language=language)
    text2bow = Text2BowTransformerPhrases(tokenizer=tokenizer, form_phrases=False)

    print('Starting bow creation')

    corpus_bow = text2bow.fit_transform(X)
    df['bow'] = corpus_bow

    print('End bow creation')

    return df

In [32]:
tokenizer = Tokenizer(tokenizer_model='stanza', language='en')
text2bow = Text2BowTransformerPhrases(tokenizer=tokenizer, form_phrases=False)

results = []
count = 0

for ind, row in df.iterrows():
    results.append(text2bow.fit_transform(row.cleaned_tweets))
    count +=1
    if count % 10 == 0:
        print(count)

10


In [33]:
df['bow'] = results
df.head()

/var/folders/79/vrkgvtgs11v73pnty27_x1100000gn/T/ipykernel_76027/1733204128.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bow'] = results


,tweets,cleaned_tweets,crypto_sentiment,subjectivity,polarity,sentiment,target,bow
date_clean,,,,,,,,
2021-10-28,"Airdrop is live , dont be late sir :)\n\n\n\n\...",Airdrop live dont late sir Airdrop Airdrops Ai...,negative,0.550000,-0.081818,negative,True,"[[], [], [], [], [], [], [], [], [], [], [], [..."
2021-10-22,something Big is coming \n$KLV Klever 💜🌟\n\nBT...,something Big coming KLV Klever BTC ETH BNB TR...,negative,0.100000,0.000000,positive,False,"[[], [], [], [], [], [], [], [], [], [], [], [..."
2021-08-16,Easy money with \n\n bsc bnb BinanceSmartChain...,Easy money bsc bnb BinanceSmartChain binance g...,negative,0.833333,0.433333,neutral,False,"[[], [], [], [], [], [], [], [], [], [], [], [..."
2021-10-19,NanoDogecoin is the King👑of the BSC Blockchain...,NanoDogecoin King BSC Blockchain DeFi altcoins...,negative,0.733333,0.433333,neutral,False,"[[(0, 1)], [], [(0, 1)], [(1, 1)], [], [(1, 1)..."
2021-12-30,"Bitcoin News Roundup for June 17, 2020 cryptoc...",Bitcoin News Roundup June cryptocurrencies bit...,positive,0.000000,0.000000,positive,True,"[[], [], [], [], [], [], [], [], [], [], [], [..."


In [38]:
df.to_csv(f'{directory}/data/tweets_nlp_bow.csv')